# Mission to Mars
## The results of the notebook are printed at the end under "Results and Print outs" section

### Get data from various Mars information pages to plan for an upcoming trip

#### Mars News from NASA
#### Highlighted picture from NASA JPL
#### Mars Weather from Twitter
#### Mars facts from space-fact.com
#### Titles and graphic URL from usgs.gov

In [1]:
#Import Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
import requests
import os

In [2]:
# Sets URL's to be used throughout the process to variables
mars_news_url = 'https://mars.nasa.gov/news/'
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
mars_tweet_url = 'https://twitter.com/marswxreport?lang=en'
mars_facts_url = 'https://space-facts.com/mars/'
mars_hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

#Sets variables for the chromedrive tool
executable_path = {'executable_path': 'e:\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News
#### Get html data from NASA using the request module
#### Use beautiful soup to parse the data and store to a variable
#### Evaluate HTLM to find news titles and paragraph information

In [3]:
# Retrieve page with the requests module
news_response = requests.get(mars_news_url)
# Create BeautifulSoup object; parse with 'lxml'
news_soup = bs(news_response.text, 'lxml')

In [4]:
# Store each div slide class into an array for evalaution 
results = news_soup.find_all('div', class_='slide')
news_title_p = []

# Loop through each section of the data to capture the news title and paragraph information
for result in results:
    try:
        news_title = result.find('div', class_='content_title').text
        news_title = news_title.strip('\n')
        news_p = result.find('div', class_='rollover_description_inner').text
        news_p = news_p.strip('\n')
        news_load = {"news_title":news_title, "news_p": news_p}
        news_title_p.append(news_load)
    except Exception as e:
        print(e)

# JPL Mars Space Images - Featured Image
#### Use Chromedriver to open the NASA JPL page
#### User splinter to nagvigate to the image details page
#### Evaluate HTLM to find link and set to feature_image_url

In [5]:
browser.visit(jpl_url)

In [6]:
jpl_html = browser.html
jpl_soup = bs(jpl_html, 'html.parser')

In [7]:
browser.click_link_by_partial_text('FULL IMAGE')

C:\Users\skiph\anaconda3\envs\BootCamp1\lib\site-packages\splinter\driver\webdriver\__init__.py:525: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [8]:
browser.click_link_by_partial_text('more info')

In [9]:
fig_html = browser.html
fig_soup = bs(fig_html, 'html.parser')

In [10]:
img_link = fig_soup.find('figure', class_='lede')
featured_image_url = "https://www.jpl.nasa.gov" + img_link.a['href']

# Mars Weather from Twitter

#### Use Chromedriver to open Mars Weather twitter page
#### Evaluate HTLM to find tweets
#### Check tweets to determine if they contain sol
#### if they do, write to attribute and print

In [11]:
browser.visit(mars_tweet_url)

In [12]:
weather_html = browser.html
weather_soup = bs(weather_html, 'html.parser')

In [13]:
tweets = weather_soup.find_all('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")

In [14]:
for tweet in tweets:
    if 'sol' in tweet.text:
        mars_weather = tweet.text
        break
    else: 
        pass

# Mars Facts

## Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

## Use Pandas to convert the data to a HTML table string.

#### Use Chromedriver to open Mars Weather twitter page
#### Evaluate HTLM to find tweets
#### Check tweets to determine if they contain sol
#### if they do, write to attribute and print


In [15]:
mars_fact_table = pd.read_html(mars_facts_url)

In [16]:
mars_facts_df = mars_fact_table[0]
mars_facts_df.columns = ['Description','Value']

In [17]:
facts_html = mars_facts_df.to_html()

# Mars Hemispheres

### Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

#### It seems the page has changed since the assignment was created. 
#### Use Chromedriver to open the Mars Hemisphere usgs page
#### Created loop to capture each of the pages that contain hemisphere information
#### Evaluate the html to pull the link to the full size picture and title of the hemisphere (title had to be modified to remove ' enhanced'

In [18]:
hemi_response = requests.get(mars_hemi_url)
hemi_soup = bs(hemi_response.text, 'html.parser')

In [19]:
hemi_links = hemi_soup.find_all('div', class_='item')

In [20]:
hemi_img_links = []

for link in hemi_links:
    hemi_url = link.find('a')['href']
    long_img_url = ('https://astrogeology.usgs.gov' + hemi_url)
    hemi_img_links.append(long_img_url)

In [21]:
hemisphere_image_urls = []

for link in hemi_links:
    hemi_url = link.find('a')['href']
    browser.visit('https://astrogeology.usgs.gov' + hemi_url)
    temp_title_html = browser.html
    soup_hemi_title = bs(temp_title_html, 'html.parser')
    img_title = soup_hemi_title.find('h2', class_='title').text
    img_title = img_title[:-9]
    img_url_large = soup_hemi_title.find('div', class_='downloads').find('a')['href']
    hemi_load = {"image title":img_title, "image url": img_url_large}
    hemisphere_image_urls.append(hemi_load)

# Results and Print outs

### Mars News

In [22]:
for each in news_title_p:
    print('----------------------')
    print(each)

----------------------
{'news_title': "The Launch Is Approaching for NASA's Next Mars Rover, Perseverance", 'news_p': "The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. "}
----------------------
{'news_title': 'NASA to Hold Mars 2020 Perseverance Rover Launch Briefing', 'news_p': "Learn more about the agency's next Red Planet mission during a live event on June 17."}
----------------------
{'news_title': "Alabama High School Student Names NASA's Mars Helicopter", 'news_p': "Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet."}
----------------------
{'news_title': "Mars Helicopter Attached to NASA's Perseverance Rover", 'news_p': "The team also fueled the rover's sky crane to get ready for this summer's history-making launch."}
----------------------
{'news_title': "NASA's Perseverance Mars Rover

### Featured URL

In [23]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA10181_hires.jpg'

### Mars Weather

In [24]:
print(mars_weather)

InSight sol 564 (2020-06-28) low -89.3ºC (-128.8ºF) high -4.3ºC (24.2ºF)
winds from the SSW at 5.0 m/s (11.3 mph) gusting to 15.4 m/s (34.4 mph)
pressure at 7.70 hPa


###  Mars Facts Dataframe and created html

In [25]:
mars_facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [26]:
facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 

### Mars image links and created title/url array

In [27]:
for each in hemi_img_links:
    print('----------------------')
    print(each)

----------------------
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
----------------------
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
----------------------
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
----------------------
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [28]:
for each in hemisphere_image_urls:
    print('----------------------')
    print(each)

----------------------
{'image title': 'Cerberus Hemisphere', 'image url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
----------------------
{'image title': 'Schiaparelli Hemisphere', 'image url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
----------------------
{'image title': 'Syrtis Major Hemisphere', 'image url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
----------------------
{'image title': 'Valles Marineris Hemisphere', 'image url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}
